## Objectives
* Solve some real world questions related to IPL 2018 using numpy and pandas functions and structure

* Questions : 
  * What is the max score of IPL 2018 and who scored it?
  * How many centuries were hit? 
  * List down the names, scores and strike rates of batsmen making centuries
  * List down top 5 most consistent batsmen

In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('data/iplfinal.csv',sep='|')
df.head()

,match_description,match_date,match_venue,match_location,match_result,team_name,innings_order,batsman_name,dismissal_mode,runs,balls,fours,sixes
0,"Mumbai Indians vs Chennai Super Kings, 1st Mat...",2018-04-07,"Wankhede Stadium,",Mumbai,Chennai Super Kings won by 1 wkt,Mumbai Indians Innings,innings_1,Rohit Sharma (c),c Rayudu b Watson,15,18,1,1
1,"Mumbai Indians vs Chennai Super Kings, 1st Mat...",2018-04-07,"Wankhede Stadium,",Mumbai,Chennai Super Kings won by 1 wkt,Mumbai Indians Innings,innings_1,Evin Lewis,lbw b D Chahar,0,2,0,0
2,"Mumbai Indians vs Chennai Super Kings, 1st Mat...",2018-04-07,"Wankhede Stadium,",Mumbai,Chennai Super Kings won by 1 wkt,Mumbai Indians Innings,innings_1,Ishan Kishan (wk),c Mark Wood b Tahir,40,29,4,1
3,"Mumbai Indians vs Chennai Super Kings, 1st Mat...",2018-04-07,"Wankhede Stadium,",Mumbai,Chennai Super Kings won by 1 wkt,Mumbai Indians Innings,innings_1,Suryakumar Yadav,c Harbhajan b Watson,43,29,6,1
4,"Mumbai Indians vs Chennai Super Kings, 1st Mat...",2018-04-07,"Wankhede Stadium,",Mumbai,Chennai Super Kings won by 1 wkt,Mumbai Indians Innings,innings_1,Hardik Pandya,not out,22,20,2,0


In [76]:
#what is the maximum score of IPL 2018 and who scored it?
x = df.runs.agg(np.max)
df.loc[df.runs==x,'batsman_name':]

,batsman_name,dismissal_mode,runs,balls,fours,sixes,str_rate,out_status
680,Rishabh Pant (wk),not out,128,63,1,7,2.031746,0


In [20]:
#how many centuries were hit in IPL 2018?
df[df.runs>=100].count()

match_description    5
match_date           5
match_venue          5
match_location       5
match_result         5
team_name            5
innings_order        5
batsman_name         5
dismissal_mode       5
runs                 5
balls                5
fours                5
sixes                5
dtype: int64

In [77]:
#list down the century makers, their scores and their strike rates
df['str_rate'] = df['runs']/df['balls']
df.loc[df.runs>=100,['batsman_name','runs','balls','str_rate']].sort_values(by='runs',ascending=1)

,batsman_name,runs,balls,str_rate
739,Ambati Rayudu,100,62,1.612903
274,Chris Gayle,104,63,1.650794
285,Shane Watson,106,57,1.859649
959,Shane Watson,117,57,2.052632
680,Rishabh Pant (wk),128,63,2.031746


In [75]:
#list down the top 5 most consistent batsmen
#find out the cases where a batsman is not out or did not bat
df['out_status'] = df.dismissal_mode.map(lambda x:0 if (x=='not out'or x=='did not bat') else 1)
#create group on batsman name
grp_player = df.groupby('batsman_name')
#aggregate the runs and out_status columns
df1 = grp_player['runs','out_status'].agg(np.sum)
#calculate average
df1['average'] = df1.runs/df1.out_status
#print top 5 batsmen (exclude the cases where total count of out_cases is 0)
df1[df1['out_status']!=0].sort_values(by='average',ascending=0).head(5)

,runs,out_status,average
batsman_name,,,
MS Dhoni (c & wk),455,6,75.833333
Chris Morris,73,1,73.000000
Abhishek Sharma,63,1,63.000000
Harshal Patel,60,1,60.000000
Jos Buttler (wk),548,10,54.800000


In [8]:
# what are the total number of duck dismissals?
df[df.runs==0].count()

match_description    103
match_date           103
match_venue          103
match_location       103
match_result         103
team_name            103
innings_order        103
batsman_name         103
dismissal_mode       103
runs                 103
balls                103
fours                103
sixes                103
dtype: int64

In [30]:
# lit top 5 players by contribution of boundaries to total scores
df['boundary_runs'] = df['fours']*4 + df['sixes']*6
df.groupby('batsman_name')
df1 = grp_runs['match_description','runs','boundary_runs'].agg({'match_description':np.size,'runs':np.sum,'boundary_runs':np.sum})
df1['boundary_contrib'] = (df1.boundary_runs/df1.runs)*100
df1.sort_values(by='runs',ascending=0).head(10)

,match_description,runs,boundary_runs,boundary_contrib
batsman_name,,,,
Kane Williamson (c),17,735,420,57.142857
Rishabh Pant (wk),15,712,460,64.606742
Lokesh Rahul (wk),15,710,428,60.281690
Ambati Rayudu,16,602,416,69.102990
Virat Kohli (c),15,561,326,58.110517
Shane Watson,15,555,346,62.342342
Jos Buttler (wk),13,548,294,53.649635
Dinesh Karthik (c & wk),17,533,308,57.786116
AB de Villiers,12,524,290,55.343511


In [78]:
# which location is most batsman friendly
# the location which has the most runs made per match + where %of runs from boundaries is high
grp_location = df.groupby('match_location')
#grp_location.groups
df1 = grp_location['runs','boundary_runs'].agg(np.sum)
df2 = grp_location.match_description.nunique()
df1['match_count'] = df2
df1['runs_per_match'] = df1.runs/df1.match_count
df1['boundary_contrib'] = df1.boundary_runs/df1.runs
df1

,runs,boundary_runs,match_count,runs_per_match,boundary_contrib
match_location,,,,,
Bengaluru,2463,1526,7,351.857143,0.619570
Chennai,395,266,1,395.000000,0.673418
Delhi,2285,1394,7,326.428571,0.610066
Hyderabad,2034,1096,7,290.571429,0.538840
Indore,1248,826,4,312.000000,0.661859
Jaipur,2054,988,7,293.428571,0.481013
Kolkata,3113,2068,9,345.888889,0.664311
Mohali,1378,796,3,459.333333,0.577649
Mumbai,2860,1722,9,317.777778,0.602098


In [80]:
df1.runs_per_match = (df1.runs_per_match - df1.runs_per_match.min())/(df1.runs_per_match.max() - df1.runs_per_match.min())
#df1.boundary_contrib = (df1.boundary_contrib - df1.boundary_contrib.min())/(df1.boundary_contrib.max() - df1.boundary_contrib.min())
df1['final_score'] = (df1.runs_per_match*.5 + df1.boundary_contrib*.5)*10
df1[df1.match_count!=1].sort_values(by='final_score',ascending=0)

,runs,boundary_runs,match_count,runs_per_match,boundary_contrib,final_score
match_location,,,,,,
Mohali,1378,796,3,1.000000,0.577649,7.888244
Kolkata,3113,2068,9,0.327784,0.664311,4.960475
Bengaluru,2463,1526,7,0.363149,0.619570,4.913593
Delhi,2285,1394,7,0.212472,0.610066,4.112687
Indore,1248,826,4,0.126975,0.661859,3.944171
Pune,1927,1152,6,0.181292,0.597820,3.895564
Mumbai,2860,1722,9,0.161211,0.602098,3.816547
Hyderabad,2034,1096,7,0.000000,0.538840,2.694199
Jaipur,2054,988,7,0.016930,0.481013,2.489713
